In [3]:
from data_creator import get_font_ttfs, create_alphabet_for_each_ttf

In [3]:
get_font_ttfs()

0it [00:00, ?it/s]

In [2]:
create_alphabet_for_each_ttf()

Razing
Military Poster
JMH Typewriter
No Virus
Who asks Satan
FoieCanape
Dunkirk
Seagram tfb
Babiole
OldNewspaperTypes
Short Stack
Deutsch Gothic
LaIzzIerComiC
Vogue
Espen Comics
Mandrake FF
The Kids Mraker
Canted Comic
Ambambo
Nexa
Blueto
Handwriting
Rocket Mission
a Anti Corona
Tabby - Personal Use
MiTica
gsmfont
Akira Expanded
Edson_Comics_Bold
Riesling
LeArchitect
Topuz
Coyotris Comic
Indernik_Charcoal
Nathaniel-19
Ducky
Verve
Asteristico
armfight
Koshari
Onion Rocks
Mulled Wine Season
Wash Your Hand
Edition
WOODCUTTER ARMY (Stencil)
Bodoni Fragile
OlivesLight
ByronRecCon
Komikula
LT Crafted
Fette National Fraktur
Sakura Town Demo
Knobber
DCC - Anatolia Strong
Brandon Bromley
RetroGaming
Couture
Mufferaw
DCC - Dreamer
The Alphabet
Mucky Sans
QuickAsABunny
Comic Note Smooth
Ohio Collegiate
Alphakind
King Arthur Legend
Comic Helvetic
Spork
mie aceh
AmazGoDaBold
FuturaHandwritten
Kaph
Monicacomic
Rundeck
I.F.C. HARDBALL 
Alex Toth
SnapHand
Cardigan Titling
Polsyh
Funiko
Pulang
Enchant

In [28]:
import os
import pandas as pd
import json

import uuid
from fontpreview import FontPreview
def create_dataset():
    FONT_IMAGE_PATH = os.path.join(os.getcwd(), 'font-images')
    assert os.path.exists(FONT_IMAGE_PATH)
    TTF_PATH = os.path.join(os.getcwd(), 'ttf-files')
    assert os.path.exists(TTF_PATH)
    CSV_PATH = os.path.join(os.getcwd(), 'CS395T - CV - Font Dataset - Sheet1.csv')

    TTF_FNAMES = os.listdir(TTF_PATH)
    TTF_FNAMES.sort()

    IMG_DIR_FNAMES = os.listdir(FONT_IMAGE_PATH)
    IMG_DIR_FNAMES.sort()
    
    # Step 1: Initialize the json file
    # Step 2: Loop through the Dataframe, for each row the Filename column corresponds to the actual
    #         folder name in 'font_images'.
    # Step 3: For each image in the respective folder, copy it over to the training folder (renaming it) and add its entry
    #         to the jsonl file

    #Step 1
    json_metadata = []
    # if not os.path.exists(training_data_path):
    #     os.makedirs(training_data_path)


    #Step 2
    df = pd.read_csv(CSV_PATH)
    head = df.head()
    
    file_name_counter = '0'
    idx = 0
    json_metadata = []
    # d = df{}
    for idx, row_data in df.iterrows():
        ttf_path = os.path.join(TTF_PATH, row_data['Filename'])
        font_img_dir = FontPreview(ttf_path).font.getname()[0]
        font_img_dir = os.path.join(FONT_IMAGE_PATH, font_img_dir)
        font_img_paths = [os.path.join(font_img_dir, fname) for fname in os.listdir(font_img_dir)]
        included_chars = [cur_path.split('/')[-1].split('.')[0] for cur_path in font_img_paths]
        json_data_row = {
            'uniqueId': str(uuid.uuid4()),
            'font_img_paths': font_img_paths,
            'ttf_path': ttf_path,
            'font_characteristics': row_data['Descriptors'], 
            'chars': included_chars,
            'font_properties': {
                'font_weight': row_data['Weight'], 
                'rounding': row_data['Courner Rounding'], 
                'font_serifs': row_data['Serif'],
                'width': row_data['Width'],
                'capitals': row_data['Capitals'],
                'dynamics': row_data['Dynamics'] 
            }
        }
        json_metadata.append(json_data_row)
    #Create the jsonl file
    with open('metadata.jsonl', 'w') as f:
        for item in json_metadata:
            f.write(json.dumps(item) + '\n')

    

            



create_dataset()
# 000 InsomniacOvrLrd Comic Dialogue.ttf
# 000 InsomniacOvrLrd Comic Dialogue.ttf